# Indeed Web scraping 

Install necessary packages

In [9]:
#Using BeautifulSoup
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup
from time import sleep
from random import randint
import time

In [ ]:
###Test with individual job cards
def get_url(position, location):
    """Generate url from position and location"""
    template = 'https://www.indeed.com/jobs?q={}&l={}'
    position = position.replace(' ', '+')
    location = location.replace(' ', '+')
    url = template.format(position, location)
    return url

In [ ]:
url = get_url('information  science', 'Washington DC')
print(url)

In [ ]:
delay = randint(1, 10)
sleep(delay)
response = requests.get(url)

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')
cards = soup.find_all('div', 'jobsearch-SerpJobCard')
print(soup)

In [ ]:
len(cards)

In [ ]:
#Try with only 1st job card
card = cards[0]

In [ ]:
job_title = card.h2.a.get('title')
job_title

In [ ]:
company = card.find('span', 'company').text.strip()
company

In [ ]:
job_location = card.find('div', 'recJobLoc').get('data-rc-loc')
job_location

In [ ]:
post_date = card.find('span', 'date').text
today = datetime.today().strftime('%Y-%m-%d')

In [ ]:
summary = card.find('div', 'summary').text.strip().replace('\n', ' ')
summary

In [ ]:
job_url = 'https://www.indeed.com' + card.h2.a.get('href')
job_url

In [ ]:
responsenew = requests.get(job_url)
responsenew

In [ ]:
soup = BeautifulSoup(responsenew.text, 'html.parser')
job_desc = soup.find("div", id="jobDescriptionText").text
job_desc

In [ ]:
record = (job_title, company, job_location, post_date, today, summary, job_url, job_desc)
record

In [ ]:
#Combining above elements
def get_url(position, location):
    """Generate url from position and location"""
    template = 'https://www.indeed.com/jobs?q={}&l={}'
    position = position.replace(' ', '+')
    location = location.replace(' ', '+')
    url = template.format(position, location)
    return url

# Get Job Description text
def get_job_desc(job_url):
    time.sleep(20)
    job_details = requests.get(job_url)
    detail_soup = BeautifulSoup(job_details.text, 'html.parser')

    job_desc = detail_soup.find('div','jobsearch-jobDescriptionText').text.strip()
    return job_desc

#Job details      
def get_record(card):
    """Extract job data from a single record"""
    try:
        job_title = card.h2.a.get('title')
    except AttributeError:
        job_title = ''
    try:
        company = card.find('span', 'company').text.strip()
    except AttributeError:
        company = ''
    try:
        job_location = card.find('div', 'recJobLoc').get('data-rc-loc')
    except:
        job_location = ''

    try:
        post_date = card.find('span', 'date').text
    except:
        post_date = ''
        
    try:
        summary = card.find('div', 'summary').text.strip().replace('\n', ' ')
    except:
        summary = ''
        
    today = datetime.today().strftime('%Y-%m-%d')
    job_desc_url = 'https://www.indeed.com' + card.h2.a.get('href')
    job_desc = get_job_desc(job_desc_url)
    record = (job_title, company, job_location, post_date, summary,today,job_desc_url, job_desc)
    return record


def main(position, location):
    """Run the main program routine"""
    records = []
    url = get_url(position, location)
    print("Starting to scrape indeed for '{}' in '{}'".format(position, location))
    
    # extract the job data
    while True:
        time.sleep(50) # limit request using sleep
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'jobsearch-SerpJobCard')
#         print(cards[0])
    
        for card in cards:
            record = get_record(card)
            # Append the results
            records.append(record)
        
        #Loop over other pages
        try:
            url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
            delay = randint(1, 10)
            sleep(delay)
        except AttributeError:
            break
    print('Finished collecting job postings')
        
    # save the job data
    with open('inforesults6', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['JobTitle', 'Company', 'Location', 'PostDate', 'Summary', 'ExtractDate', 'JobUrl','JobDesc'])
        writer.writerows(records)

In [11]:
# run the main program
main('information science ', 'Washington DC')

Starting to scrape indeed for 'information science ' in 'Washington DC'
Finished collecting job postings


In [3]:
import pandas as pd
info_science = pd.read_csv('inforesults6.csv')
# Displays whole table
# data_science
info_science


,JobTitle,Company,Location,PostDate,Summary,ExtractDate,JobUrl,JobDesc
0,Youth Development Representative,Government of the District of Columbia,"Washington, DC",13 days ago,Youth Development Representatives are also obl...,2021-05-02,https://www.indeed.com/rc/clk?jk=7300207c008e2...,The DC Department of Youth Rehabilitation Serv...
1,Research Associate (Political Economy),AGSIW,"Washington, DC",1 day ago,"ABDs and doctoral candidates in economics, int...",2021-05-02,https://www.indeed.com/rc/clk?jk=b7376e9627cd3...,Department/Team: Resident Scholars\n\nPeriod: ...
2,Program Assistant: Eurasia (Multiple Vacancies),National Democratic Institute,"Washington, DC",5 days ago,High level of oral and written communication s...,2021-05-02,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Job Function: Washington DC\nDepartment: Euras...
3,Washington Junior Government Affairs Analyst,K&L Gates,"Washington, DC",2 days ago,The candidate will serve as a client and proje...,2021-05-02,https://www.indeed.com/rc/clk?jk=75e1e9b379802...,"Job Description\nAt K&L Gates, we are looking ..."
4,Associate Political Officer,Foreign & Commonwealth Office,"Washington, DC",12 days ago,Bachelor’s Degree in a relevant field such as ...,2021-05-02,https://www.indeed.com/rc/clk?jk=6eff3a0723b4c...,Job Description (Roles and Responsibilities)\n...
...,...,...,...,...,...,...,...,...
1062,Patent Attorney / Agent / Electrical Software ...,IPRecruiter Group,"Washington, DC",30+ days ago,"Relevant Terms: patent, IP, intellectual prope...",2021-05-02,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Patent Attorney or Agent opportunity within an...
1063,Principal/Architect Data Science/ Machine Lear...,Salesforce,"Washington, DC",30+ days ago,Help designing experiments through agile data ...,2021-05-02,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,"To get the best candidate experience, please c..."
1064,Senior DevOps Engineer,Talascend LLC,"Herndon, VA",16 days ago,"The Senior Engineer, DevOps works as part of a...",2021-05-02,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Talascend is currently seeking a Senior DevOps...
1065,Technical Program Manager (Data Engineering),Belcan,"Herndon, VA",30+ days ago,A remote-based Technical Program Manager based...,2021-05-02,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,A remote-based Technical Program Manager based...


In [4]:
info_science.to_csv('/Users/neviyaprakash/Desktop/UMD MIM/Spring 2021/INFM 737 Capstone/info_Sci.csv')

Method 2 With Selenium

In [16]:
from selenium import webdriver
import pandas as pd 
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait

In [17]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(executable_path = "/Applications/chromedriver")
driver.maximize_window()
wait = WebDriverWait(driver, 20)
#jobs = driver.find_elements_by_class_name('jobsearch-SerpJobCard')
df = pd.DataFrame(columns=["Title","Location","Company","Post_Date","Salary","Summary","Job URL","Description"])

for i in range(0,10,10):
    #driver.get('https://www.indeed.co.in/jobs?q=artificial%20intelligence&l=India&start='+str(i))
    driver.get('https://www.indeed.com/jobs?q=data+science&l=Washington%2C+DC&start='+ str(i))
    #jobs = []
    driver.implicitly_wait(10)
    all_jobs = driver.find_elements_by_class_name('jobsearch-SerpJobCard')  #driver.find_elements_by_class_name('result')
    
    for job in all_jobs:
#         wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'jobsearch-SerpJobCard')))
        soup = BeautifulSoup(job.get_attribute('innerHTML'),'html.parser')
        atag = soup.h2.a

        try:
            title = soup.find("a",class_="jobtitle").text.replace("\n","").strip()

        except:
            title = 'None'
        
        try:
            location = soup.find(class_="location").text
        except:
            location = 'None'

        try:
            company = soup.find(class_="company").text.replace("\n","").strip()
        except:
            company = 'None'
        try:
            post_date = soup.find(class_="date").text
        except:
            post_date = 'None'

        try:
            salary = soup.find(class_="salary").text.replace("\n","").strip()
        except:
            salary = 'None'
            
        try:
            summary = soup.find(class_="summary").text.replace("\n","").strip()
        except:
            summary = 'None'
        job_url = 'https://www.indeed.com' + atag.get('href')
        

        sum_div = job.find_element_by_xpath('./div[3]')
        #sum_div = job.find_element_by_xpath('//div[@id="jobDescriptionText"]')
        try:
            sum_div.click()
        except:
            close_button = driver.find_elements_by_class_name('popover-x-button-close')[0] # to handle job notification popup
            close_button.click()
            sum_div.click()
        #job_desc = driver.find_element_by_id('vjs-desc').text
        try:
            #wait.until(EC.presence_of_element_located((By.ID, 'vjs-content')))
#             job.click()
#             wait.until(EC.presence_of_element_located((By.ID, 'vjs-content')))
            job_desc = driver.find_element_by_id('vjs-content').text
            #job_desc = driver.find_element_by_xpath('//div[@id="jobDescriptionText"]').textdescriptions
            #soup.find("div", id="jobDescriptionText").text
            
            #job_desc = soup.find("div", id="jobDescriptionText").text
            #job_desc = driver.find_element_by_id('vjs-desc').text
            #job_desc = driver.find_element_by_css_selector('div#vjs-desc').text
            #job_desc = driver.find_element_by_id('jobDescriptionText').text
            
            #print(jd)
        except:
            job_desc = 'None'

        df = df.append({'Title':title,'Location':location,"Company":company,"Post_Date":post_date,"Salary":salary,"Summary": summary,"Job URL":job_url, "Description":job_desc},ignore_index=True)

        print("Got these many results:",df.shape)



df.to_csv("datascience.csv",index=False)



Note 
* The Selenium method did not provide the complete job description text, only the job summary.
* The Beautiful Soup method did give us the job description